# Extração de Texto em Imagens

* Neste Caderno é feita a conversão de arquivos PDF baseados em imagens para .png, arquivos escaneados e posteriormente salvos em formato de pdf não possuem texto alocado no conteúdo do PDF para ser extraido com um framework simples de PDF, logo, é necessária a conversão desses arquivos para um formato de imagem e em seguida é utilizado uma lib de detecção de texto em imagem.

#### Libs Utilizadas

In [1]:
!pip install -q easyocr

In [2]:
!pip install -q opencv-python

In [3]:
!pip install -q pymupdf

In [4]:
# Libs
import os # SO organização de diretorios e salvamento de arquivos.
import re # regex para limpeza do texto
import cv2 # imagem
import easyocr # imagem
import numpy as np
import pymupdf# conversão de formato de arquivos.

### 

In [5]:
import os
import fitz  # pymupdf
import cv2
import easyocr

# Configurações
threshold = 0.25  # Confiança mínima para o OCR
reader = easyocr.Reader(['en'], gpu=False)  # Inicialização do leitor OCR

def convert_pdf_to_images(pdf_path, output_dir, dpi=300):
    """Converte um PDF em uma série de imagens PNG."""
    pdf_document = pymupdf.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pix = page.get_pixmap(dpi=dpi)
        
        # Cria o caminho completo para a imagem
        output_path = os.path.join(output_dir, f"paginaTeste_{page_num + 1}.png")
        pix.save(output_path)
        image_paths.append(output_path)
        print(f"Página {page_num + 1} salva como {output_path}")
    
    pdf_document.close()
    return image_paths

def extract_text_from_image(image_path, dest_dir_images):
    """Extrai texto de uma imagem e salva a imagem com bounding boxes."""
    img = cv2.imread(image_path)
    text_data = reader.readtext(img)
    extracted_text = ""

    for bbox, text, score in text_data:
        if score > threshold:
            extracted_text += f"{text}\n"
            # Desenha a bounding box e o texto na imagem
            cv2.rectangle(img, tuple(map(int, bbox[0])), tuple(map(int, bbox[2])), (0, 255, 0), 2)
            cv2.putText(img, text, tuple(map(int, bbox[0])), cv2.FONT_HERSHEY_COMPLEX, 0.65, (255, 0, 0), 2)

    # Salva a imagem com as anotações
    annotated_image_path = os.path.join(dest_dir_images, os.path.basename(image_path))
    cv2.imwrite(annotated_image_path, img)
    print(f"Imagem processada salva em: {annotated_image_path}")

    return extracted_text.strip()

# Inserir caminho completo do PDF
pdf_path = 'C:/Users/Lucas Silva/Documents/leitura/aler/Purchase Order.pdf'

# Diretório de saída
output_dir = os.path.dirname(pdf_path)  # Usa o mesmo diretório do PDF
dest_dir_images = os.path.join(output_dir, 'processed_images')
os.makedirs(dest_dir_images, exist_ok=True)  # Cria o diretório se não existir

# Pipeline
print("Convertendo PDF em imagens...")
image_paths = convert_pdf_to_images(pdf_path, output_dir)

print("\nExtraindo texto das imagens...")
all_extracted_text = ""
for image_path in image_paths:
    extracted_text = extract_text_from_image(image_path, dest_dir_images)
    all_extracted_text += f"{extracted_text}\n\n"

# Resultado Final
print("\nTexto extraído de todas as páginas:")
print(all_extracted_text)


Using CPU. Note: This module is much faster with a GPU.


Convertendo PDF em imagens...
Página 1 salva como C:/Users/Lucas Silva/Documents/leitura/aler\paginaTeste_1.png

Extraindo texto das imagens...
Imagem processada salva em: C:/Users/Lucas Silva/Documents/leitura/aler\processed_images\paginaTeste_1.png

Texto extraído de todas as páginas:
Republic of the Philippines
(Formerly Ramon Magsaysay Technological University )
Iba Zambales
TellFax No:: (047) 811-1683
PURCHASE ORDER
Supplier: PHILCOPY CORPORATION
Purchase Order No:: 2024-02-017
Address: Rizal St , Makati
Date: February 19,2024
Mode of Procurement: Direct Contracting
TIN Number: 000-169-318-000
Gentlemen:
Please furnish this Office the following articles subject to the terms and condition contained herein:
Purpose: "Supply and Delivery of Office Supplies"
For Office ofthe Campus Registrar and Office ofthe University Admission and Registration Services Use;
Place of Delivery: Supply Office-Engr T Faustino(Edgar R Pastores, MPA)
Delivery Term: DAP
Date of Delivery: March
1 ,
2024
Pay